<a href="https://colab.research.google.com/github/Ismail-therap/NLP-Project-Predicting-review-in-1-to-5-star-/blob/main/CNN_and_Hybrid_of_CNN%2BLSTM_in_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importing libraries
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [ ]:
# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
link2 = 'https://drive.google.com/open?id=1cfhUjGNvEBBxAXCFX8XihCULg5HAIE1v'

fluff, id2 = link2.split('=')
print (id2) # Verify that you have everything after '='

1cfhUjGNvEBBxAXCFX8XihCULg5HAIE1v


In [ ]:
import pandas as pd
downloaded2 = drive.CreateFile({'id':id2}) 
downloaded2.GetContentFile('yelp_review_partial.csv') 
data = pd.read_csv('yelp_review_partial.csv')
data.head()

,Unnamed: 0,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,0,lWC-xP3rd6obsecCYsGZRg,ak0TdVmGKo4pwqdJSTLwWw,buF9druCkbuXLX526sGELQ,4,3,1,1,Apparently Prides Osteria had a rough summer a...,2014-10-11 03:34:02
1,1,8bFej1QE5LXp4O05qjGqXA,YoVfDbnISlW0f7abNQACIg,RA4V8pr014UyUbDvI-LW2A,4,1,0,0,This store is pretty good. Not as great as Wal...,2015-07-03 20:38:25
2,2,NDhkzczKjLshODbqDoNLSg,eC5evKn1TWDyHCyQAwguUw,_sS2LBIGNT5NQb6PD1Vtjw,5,0,0,0,I called WVM on the recommendation of a couple...,2013-05-28 20:38:06
3,3,T5fAqjjFooT4V0OeZyuk1w,SFQ1jcnGguO0LYWnbbftAA,0AzLzHfOJgL7ROwhdww2ew,2,1,1,1,I've stayed at many Marriott and Renaissance M...,2010-01-08 02:29:15
4,4,sjm_uUcQVxab_EeLCqsYLg,0kA0PAJ8QFMeveQWHFqz2A,8zehGz9jnxPqXtOc7KaJxA,4,0,0,0,The food is always great here. The service fro...,2011-07-28 18:05:01


# Text processing for CNN and (CNN+LSTM)

In [ ]:
from sklearn.model_selection import train_test_split
data['cnn_star'] = data['stars'] - 1
review = data['text'].values
label = data['cnn_star'].values
review_train, review_test, label_train, label_test = train_test_split(review, label, test_size=0.25, random_state=1000)

num_classes = 5
at_label_train = k.utils.to_categorical(label_train, num_classes)
at_label_test = k.utils.to_categorical(label_test, num_classes)


from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(review_train)
Xcnn_train = tokenizer.texts_to_sequences(review_train)
Xcnn_test = tokenizer.texts_to_sequences(review_test)
vocab_size = len(tokenizer.word_index) + 1  


from keras.preprocessing.sequence import pad_sequences
maxlen = 450
Xcnn_train = pad_sequences(Xcnn_train, padding='post', maxlen=maxlen)
Xcnn_test = pad_sequences(Xcnn_test, padding='post', maxlen=maxlen)
print(Xcnn_train[0, :]) 

[ 469   21    4  248  291   25   37   10   12  108    6  137   14   16
  297   33  200   31    5 1832   52 2237 2238   11   25  314    4 2849
   24    3  171 1833    3   87   11   33    2   12 2850    3   76  200
   31    5 1832   37    7 4136   35 4137   40   62 4138    2   20  701
 1550 1551 2851   20   46   15  886    1  286  113   53   60    3 2239
   16  102  629   33   52  426  136 1552   53   60    8  151   19    1
 2240 2852   24  190  132  201    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0 

# Convulation Neural Network Model (CNN)

In [ ]:
embedding_dim = 100
textcnnmodel = Sequential()
textcnnmodel.add(layers.Embedding(vocab_size, embedding_dim, input_length=maxlen))
textcnnmodel.add(layers.Conv1D(128, 5, activation='relu',padding = 'same'))
textcnnmodel.add(layers.GlobalMaxPooling1D())
textcnnmodel.add(layers.Flatten())
textcnnmodel.add(layers.Dense(5, activation='softmax'))
textcnnmodel.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
textcnnmodel.fit(Xcnn_train, label_train,
                     epochs=10,
                     validation_data=(Xcnn_test, label_test),
                     batch_size = 64)
loss, accuracy = textcnnmodel.evaluate(Xcnn_train, label_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = textcnnmodel.evaluate(Xcnn_test, label_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

Epoch 1/10
12/12 [==============================] - 7s 556ms/step - loss: 1.5578 - accuracy: 0.2680 - val_loss: 1.4826 - val_accuracy: 0.3720
Epoch 2/10
12/12 [==============================] - 6s 538ms/step - loss: 1.3679 - accuracy: 0.4400 - val_loss: 1.4834 - val_accuracy: 0.3480
Epoch 3/10
12/12 [==============================] - 6s 536ms/step - loss: 1.2927 - accuracy: 0.4080 - val_loss: 1.4624 - val_accuracy: 0.3480
Epoch 4/10
12/12 [==============================] - 6s 535ms/step - loss: 1.2079 - accuracy: 0.5933 - val_loss: 1.4526 - val_accuracy: 0.3480
Epoch 5/10
12/12 [==============================] - 6s 534ms/step - loss: 1.1113 - accuracy: 0.6133 - val_loss: 1.4436 - val_accuracy: 0.3480
Epoch 6/10
12/12 [==============================] - 6s 533ms/step - loss: 0.9973 - accuracy: 0.6720 - val_loss: 1.4239 - val_accuracy: 0.3640
Epoch 7/10
12/12 [==============================] - 6s 533ms/step - loss: 0.8657 - accuracy: 0.7653 - val_loss: 1.4069 - val_accuracy: 0.3640
Epoch 

# Hybrid (CNN + LSTM)

In [ ]:
import h5py as h5
import scipy.io as sio
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from os import listdir
from tensorflow.keras.preprocessing import sequence
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Embedding, Dropout, Activation, Reshape
from tensorflow.keras.layers import Dense, Conv1D, MaxPooling1D, GlobalAveragePooling1D, TimeDistributed
from tensorflow.keras.layers import LSTM
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ModelCheckpoint
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score


In [ ]:
embedding_dim = 100
model = Sequential()
model.add(layers.Embedding(vocab_size, embedding_dim, input_length=maxlen))
model.add(Conv1D(filters=128,kernel_size = 5,padding = 'valid',activation = 'relu', strides = 1))
model.add(MaxPooling1D(pool_size = 10))
model.add(Conv1D(filters=64,kernel_size = 5,padding = 'valid',activation = 'relu', strides = 1))
#model.add(LSTM(100,return_sequences = True,activation='sigmoid',recurrent_activation = 'sigmoid',dropout=0.2,recurrent_dropout=0.2))
#model.add(LSTM(100,return_sequences = True,activation='sigmoid',recurrent_activation = 'sigmoid',dropout=0.2,recurrent_dropout=0.2))
model.add(LSTM(units=200))
model.add(Dense(5,activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy',optimizer = 'adam',metrics = ['accuracy'])

In [ ]:
model.fit(Xcnn_train, label_train,
                     epochs=10,
                     validation_data=(Xcnn_test, label_test),
                     batch_size = 64)
loss, accuracy = model.evaluate(Xcnn_train, label_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(Xcnn_test, label_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

Epoch 1/10
12/12 [==============================] - 8s 498ms/step - loss: 1.5002 - accuracy: 0.3907 - val_loss: 1.4932 - val_accuracy: 0.3040
Epoch 2/10
12/12 [==============================] - 5s 449ms/step - loss: 1.4578 - accuracy: 0.3467 - val_loss: 1.4934 - val_accuracy: 0.3480
Epoch 3/10
12/12 [==============================] - 5s 451ms/step - loss: 1.4435 - accuracy: 0.4000 - val_loss: 1.4824 - val_accuracy: 0.3480
Epoch 4/10
12/12 [==============================] - 5s 452ms/step - loss: 1.4366 - accuracy: 0.4027 - val_loss: 1.4720 - val_accuracy: 0.3480
Epoch 5/10
12/12 [==============================] - 5s 452ms/step - loss: 1.4204 - accuracy: 0.3960 - val_loss: 1.5124 - val_accuracy: 0.3480
Epoch 6/10
12/12 [==============================] - 5s 450ms/step - loss: 1.3932 - accuracy: 0.4053 - val_loss: 1.5201 - val_accuracy: 0.3480
Epoch 7/10
12/12 [==============================] - 5s 452ms/step - loss: 1.2608 - accuracy: 0.4440 - val_loss: 1.3945 - val_accuracy: 0.4080
Epoch 